# Create a Grain Boundary with Strain Matching

Create a grain boundary between two crystallographic orientations of the same material using strain matching algorithm to minimize interfacial strain.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
2. Set notebook parameters in cell 1.1 below (or use the default values).
3. Set slab parameters for both phases in cell 2.1 (or use default).
4. Set grain boundary parameters in cell 3.1 (or use default).
5. Click "Run" > "Run All" to run all cells.
6. Wait for the run to complete (depending on the parameters can take a few minutes).
7. Scroll down to view results.

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input material
2. Create slabs for both phases and select the terminations
3. Generate grain boundaries with strain matcher and plot strain vs number of atoms
4. Select the grain boundary with the desired strain and visualize it

## Notes
1. We perform strain matching on the slabs to extract the supercell dimensions.
2. When the strain matching is finished, the grain boundary with the lowest strain (and reasonable number of atoms) is selected.
3. For more information about grain boundaries, see the Introduction notebook.


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [ ]:
# Parameters for Phase 1
PHASE_1_MILLER_INDICES = (1, 1, 1)
PHASE_1_THICKNESS = 2  # in atomic layers
PHASE_1_TERMINATION_FORMULA = None  # if None, the first termination will be used
PHASE_1_USE_ORTHOGONAL_C = True

# Parameters for Phase 2
PHASE_2_MILLER_INDICES = (0, 0, 1)
PHASE_2_THICKNESS = 2  # in atomic layers
PHASE_2_TERMINATION_FORMULA = None  # if None, the first termination will be used
PHASE_2_USE_ORTHOGONAL_C = True

INTERPHASE_GAP = 2.0  # in Angstrom
TRANSLATION_VECTOR = [0.0, 0.0, 0.0]  # Translation vector for the phase 2 slab relative to phase 1 slab

# Maximum area for the superlattice search algorithm
MAX_AREA = 350  # in Angstrom^2

# index of the grain boundary to select after strain matching
MATCH_ID = 0  # if None, the grain boundary with the lowest strain will be selected

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("create_grain_boundary.ipynb")


### 1.3. Get input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview Material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([material], repetitions=[3, 3, 1], rotation="0x")

## 2. Configure slabs and select termination pair

### 2.1. Create Phase 1 and Phase 2 Slabs
Get the terminations for the two slabs of phases.

In [ ]:
from mat3ra.made.tools.helpers import get_slab_terminations

phase_1_terminations = get_slab_terminations(
    material=material,
    miller_indices=PHASE_1_MILLER_INDICES,
)

phase_2_terminations = get_slab_terminations(
    material=material,
    miller_indices=PHASE_2_MILLER_INDICES,
)

### 2.3. Visualize slabs for all possible terminations

In [ ]:
from mat3ra.made.tools.helpers import create_slab

phase_1_slabs = [create_slab(material, miller_indices=PHASE_1_MILLER_INDICES, termination_top=termination) for termination in
                 phase_1_terminations]

phase_2_slabs = [create_slab(material, miller_indices=PHASE_2_MILLER_INDICES, termination_top=termination) for termination in
                 phase_2_terminations]

phase_1_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in
                             zip(phase_1_slabs, phase_1_terminations)]
phase_2_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in
                             zip(phase_2_slabs, phase_2_terminations)]

visualize(phase_1_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")
visualize(phase_2_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")

### 2.4. Select terminations for the grain boundary phases

In [ ]:
from mat3ra.made.tools.helpers import select_slab_termination
phase_1_selected_termination = select_slab_termination(phase_1_terminations, PHASE_1_TERMINATION_FORMULA)
phase_2_selected_termination = select_slab_termination(phase_2_terminations, PHASE_2_TERMINATION_FORMULA)

## 3. Create grain boundary

In [ ]:
from mat3ra.made.tools.helpers import create_grain_boundary_planar

grain_boundary = create_grain_boundary_planar(
    phase_1_material=material,
    phase_1_miller_indices=PHASE_1_MILLER_INDICES,
    phase_2_miller_indices=PHASE_2_MILLER_INDICES,
    phase_1_thickness=PHASE_1_THICKNESS,
    phase_2_thickness=PHASE_2_THICKNESS,
    gap=INTERPHASE_GAP,
    translation_vector=TRANSLATION_VECTOR,
    max_area=MAX_AREA,
    max_area_ratio_tol=0.2,
    match_id =MATCH_ID,
)


## 4. Preview the grain boundary

In [ ]:
visualize([grain_boundary], repetitions=[1, 1, 1])
visualize([grain_boundary], repetitions=[1, 1, 1], rotation="-90x")

## 5. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(grain_boundary)